In [15]:
import pandas as pd
import numpy as np
import sklearn.preprocessing as skpp
# XGBoost
# https://xgboost.readthedocs.io/en/stable/python/python_api.html#module-xgboost.sklearn
import xgboost as xgb # must be in AMN_windows env
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict as cvp
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.multioutput import MultiOutputRegressor
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from typing import Dict, Tuple, List


PATH = "biolog_data.txt"
all_data = pd.read_csv(PATH, comment="<", encoding='cp1252', delimiter="\t")
# all_data_norm = (all_data - all_data.mean()) - all_data.std()
all_data

,Experiment Description,Data Definition,Well Substrate,Data Value,Data Submitter
0,b3066 mutant Biolog assay,OD600 growth,(-)Shikimic Acid,1.74,Blattner Laboratory
1,b3072 mutant Biolog assay,OD600 growth,(-)Shikimic Acid,0.89,Blattner Laboratory
2,b3076 mutant Biolog assay,OD600 growth,(-)Shikimic Acid,1.07,Blattner Laboratory
3,b3091 mutant Biolog assay,OD600 growth,(-)Shikimic Acid,0.78,Blattner Laboratory
4,b3093 mutant Biolog assay,OD600 growth,(-)Shikimic Acid,1.43,Blattner Laboratory
...,...,...,...,...,...
100907,b4129 mutant Biolog assay,OD600 growth,Xylitol,0.19,Blattner Laboratory
100908,b4138 mutant Biolog assay,OD600 growth,Xylitol,0.22,Blattner Laboratory
100909,b4173 mutant Biolog assay,OD600 growth,Xylitol,0.15,Blattner Laboratory
100910,b4174 mutant Biolog assay,OD600 growth,Xylitol,0.15,Blattner Laboratory


In [12]:
print(len(set(all_data['Experiment Description'])))
print(len(set(all_data['Well Substrate'])))

212
390


In [18]:
target_gene_encoder = skpp.OneHotEncoder()
which_CS_encoder = skpp.OneHotEncoder()

raw_target_gene = all_data["Experiment Description"].values.reshape(-1, 1)
raw_which_medium = all_data["Well Substrate"].values.reshape(-1, 1)

target_gene_encoder.fit(raw_target_gene)
which_CS_encoder.fit(raw_which_medium)

X_left = target_gene_encoder.transform(raw_target_gene).toarray()
X_right = which_CS_encoder.transform(raw_which_medium).toarray()

X = np.hstack((X_left, X_right))

# raw_X = all_data.iloc[:,:3].values
# enc.fit(raw_X)
# X = enc.transform(raw_X).toarray()

Y = all_data["Data Value"].values.reshape(-1, 1)
print(X.shape, Y.shape)

(100912, 602) (100912, 1)


In [19]:
LOO_split = int(Y.shape[0])

xgb_preds = []

for i in range(5):

    # XGBoost = xgb.XGBRegressor(n_estimators=600, max_depth=2, learning_rate=0.01, objective='reg:squarederror', random_state=i)
    multioutputregressor = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror'))

    xgb_pred = cvp(multioutputregressor, X, Y, cv=KFold(n_splits=10, shuffle=True, random_state=i), n_jobs = 5)
    
    xgb_preds.append(xgb_pred)

In [22]:
mean_xgb_pred = np.array(xgb_preds).mean(axis=0)
std_xgb = np.array(xgb_preds).std(axis=0)

print("Mean squared error between mean prediction and true fluxes", np.mean((mean_xgb_pred - Y)**2, axis=0)) # MSE between mean prediction (over 5 times 10-fold test sets) and true

print("Q²: ", r2_score(Y, mean_xgb_pred))

Mean squared error between mean prediction and true fluxes [0.19389984]
Q²:  0.525509846522625
